In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)



In [2]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [3]:
n_inputs = 28*28  # MNIST
# 入力の特徴量　Xの形状は(インスタンス数,特徴量数(28*28))だが、インスタンス数は不明　1ピクセルが１特徴量になっている

n_hidden1 = 300
# 隠れ層1
n_hidden2 = 100
# 隠れ層2

n_outputs = 10
# 出力層(10個に分類される)

In [4]:
import tensorflow as tf
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") # (不明なインスタンス数,28*28) のnumpy配列
y = tf.placeholder(tf.int32, shape=(None), name="y") # 1次元でインスタンス数未定のテンソル

In [5]:
# ここからは名前スコープを使用して作っていく　⇒　グラフで見やすくなる

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    # 入力は訓練データ、隠れ層1の特徴量は300個、活性化関数としてReLUを指定
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    # 入力は隠れ層1、隠れ層2の特徴量は100個、活性化関数としてReLUを指定
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    # 入力は隠れ層2、出力は10(分類結果)



In [6]:
# コスト関数の定義

# tf.nn.sparse_softmax_cross_entropy_with_logits
# 各データのクロスエントロピーを出力する。学習時の損失関数としてはバッチ内の各データのクロスエントロピーの平均を取る必要がある
# 0～クラス数-1 までの整数ラベルを渡す(今回はMNISTなので0～9) 戻り値は交差エントロピーが入った1次元テンソル
# ここで平均交差エントロピーを出すために　tf.reduce_mean　を実行している


with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [7]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [8]:
# モデル評価方法の定義
# 単純な正解率を指標とする場合

#in_top_k() 論理値を集めた1次元テンソルを返す 
# 個々のインスタンスについて最高ロジットがターゲットクラスになっているかをチェック
# 予測が正しいかを判定

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    # logits : テンソルタイプの予測結果を渡している
    # y : 正解となるラベルデータを渡している
    # 1 : 計算精度を調べるための予測精度の数を渡している
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # 集めた予測値の平均を取ることでNN全体の正解率を出している

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [12]:
n_epochs = 20
n_batches = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.9 Validation accuracy: 0.9024
1 Batch accuracy: 0.92 Validation accuracy: 0.9254
2 Batch accuracy: 0.94 Validation accuracy: 0.9374
3 Batch accuracy: 0.9 Validation accuracy: 0.9418
4 Batch accuracy: 0.94 Validation accuracy: 0.9472
5 Batch accuracy: 0.94 Validation accuracy: 0.9514
6 Batch accuracy: 1.0 Validation accuracy: 0.9546
7 Batch accuracy: 0.94 Validation accuracy: 0.961
8 Batch accuracy: 0.96 Validation accuracy: 0.9622
9 Batch accuracy: 0.94 Validation accuracy: 0.9648
10 Batch accuracy: 0.92 Validation accuracy: 0.9652
11 Batch accuracy: 0.98 Validation accuracy: 0.9668
12 Batch accuracy: 0.98 Validation accuracy: 0.9684
13 Batch accuracy: 0.98 Validation accuracy: 0.9704
14 Batch accuracy: 1.0 Validation accuracy: 0.9696
15 Batch accuracy: 0.94 Validation accuracy: 0.9718
16 Batch accuracy: 0.98 Validation accuracy: 0.9728
17 Batch accuracy: 1.0 Validation accuracy: 0.9728
18 Batch accuracy: 0.98 Validation accuracy: 0.9748
19 Batch accuracy: 0.96 Vali

In [13]:
import tensorboard as tb

In [14]:
tb.show_graph(tf.get_default_graph())

In [18]:
# 作成したモデルを使って予測してみる
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20] # テストデータから20件だけを取得
    Z = logits.eval(feed_dict={X: X_new_scaled}) # 予測した値だけを受け取る場合はlogit でよい。確率まで知りたい場合はsoftmax関数を使う
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [19]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
